# Sriram et al. (2012) Model Code <a name="top" />

# Table of Contents 
1. [Instructions](#instructions)
    1. [Parameter Optimization Against TSST Data Sets](#TSSTInstructions)
    2. [Parameter Optimization Against Basal Data Sets](#basalInstructions)
    3. [Running Without Parameter Optimization](#noOptInstructions)
2. [Imports](#imports)
3. [Parameters and Initial Conditions](#params)
4. [Put Raw Data into Arrays](#rawdata)
    1. [Plot Basal Data Sets](#plotBasalData)
    2. [Plot Nelson Data Sets](#plotNelsonData)
5. [Model Function--Includes ODE Solver](#modelfunction)
6. [Cost Function Definition](#cost)
7. [Run the Optimization](#run)
8. [Save Output to File](#saveoutput)
9. [Compute Means and Std Devations of Parameters and Output as Table](#paramtable)
10. [Plots](#plots)
11. [Figure 3 Reproduction](#fig3)
12. [Figure 4 Reproduction](#fig4)
13. [No Optimization Run](#no-opt)
14. [Run Optimizations for Multiple Patients](#runMultiple)
15. [Checking Parameter Sets Generated Externally](#checkexternal)
16. [Load Simulated Concentration Data from Files and Plot](#load-and-plot)
17. [Dependencies](#dependencies)

# Instructions <a name="instructions"></a>

## Parameter Optimization Against TSST Data Sets <a name="TSSTInstructions" />

**Note:** To quickly run a cell (or a selection of cells), use the shortcut Shift+Enter (or you can also use the button labeled "Run" in the toolbar at the top).

To run simulations with parameter optimization against TSST data, there is no need to change any cells until the heading **Run the Optimization**. Simply run all cells up to the cell below that heading.

In order to set which data set to optimize parameters against, look for the following line of code:
    
    data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]]

In order to run against a patient from the TSST data sets, simply change the list entries to reflect the patient number and subject group. The subject groups are:

- nelson.melancholicACTH & nelson.melancholicCortisol (15 patients)
- nelson.atypicalACTH & nelson.atypicalCortisol (14 patients)
- nelson.neitherACTH & nelson.neitherCortisol (14 patients)
- nelson.healthyACTH & nelson.healthyCortisol (15 patients)

You could also run against the mean of all patients cortisol and ACTH concentration values by using `nelson.ACTH[:,1]` and `nelson.cortisol[:,1]`. Or you can run against the mean of any subgroup using `nelson.<subgroup name>Cortisol_mean[:,1]` and `nelson.<subgroup name>ACTH_mean[:,1]` (for instance `nelson.melancholicCortisol_mean[:,1]` & `nelson.melancholicACTH_mean[:,1]`). 

Note that the first column in each data set is the time steps, so indexing with `[:,0]` is referring to the time. These are the values we need to set as the first (ACTH time steps) and third (cortisol time steps) indices of the `data_to_match` list.

The following are several examples of lists you could use for parameter optimization with explanations:

- `nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1]`
    - The 1st patient in the Melancholic subgroup
- `nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14]`
    - The 14th patient in the Atypical subgroup
- `nelson.healthyACTH[:,0], nelson.healthyACTH[:,2], nelson.healthyCortisol[:,0], nelson.healthyCortisol[:,2]`
    - The 2nd patient in the Healthy Control group
- `nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]`
    - The mean data set for all patients (depressed and control)
- `nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1]`
    - The mean of all control patients
    
Next, you need to decide whether you will optimize any initial conditions (ICs). This can be modified in the function `cost_fun(params)`. In the cost function, we use the first two optimized parameters in the list returned by the optimization algorithm to set the ICs we want to optimize (CRH and GR in this example):

    y0 = [params[0], y0[1], y0[2], params[1]]

We then need to pass only the remaining parameters in the list to the model, along with the updated ICs in y0:

    simData = model(params[2:], y0)
    
If you want to not optimize any ICs, you would simply comment out the two lines above in `cost_fun()`, and uncomment the line:

    #simData = model(params, y0)
    
In that case, you will likely want to change the ICs for CRH and GR, as they will stay the same for every iteration. Under the heading **Run the Optimization**, the following line should be modified to reflect the desired ICs:

    y0 = [0, data_to_match[1][0], data_to_match[3][0], 0]
    
Be sure to leave the 2nd and 3rd indices unchanged, as these set the ICs to use the initial values of the real-world data set being matched.

At this point, you are ready to run the optimization, so simply run the cells up to the heading **Save Output to File**. This may take some time, so while it is running you can move on to the next steps (if you run a cell while another is processing, it will add it to a queue).

**Note:** You also have the option of using a cost function based on the maximum distance between simulation and real-world data. Simply change SSE_cost to MAX_cost, the instructions for function arguments remain the same.

The cell directly under the heading **Save Output to File** can be changed so that the root filename matches the simulations being run. This root will be used to save all of the various data and figures generated. The current naming scheme would save the files for 5 iterations of parameter optimization against the mean data set from the Nelson with ICs for CRH and GR optimized as:

    filename_root = "sriramModel_output/nelson-patientMean-5-iterations-ICOpt"

This saves the files in a subfolder specific to this model, which helps keep files organized when running multiple models.

The next few cells create an Excel file containing all of the concentration data and optimized parameter values, and text files containing the initial conditions, parameter bounds and parameter means +- standard deviation across the 5 iterations.

The final step after saving the outputs is to plot the simulations against the real-world data. The cell under the heading **Plots** creates an instance of the Visualizer object from the VeVaPy module called visualize. This will start a dialog which asks for several inputs to generate figures as desired.

After initialization of the object, run its method `make_graphs()`, and it will generate figures using the data you have specified. There are a number of arguments that can be optionally specified for this method, and you can see the recommended values for these in the following function call:

    grapher.make_graphs(std_dev = True, 
                sims_line_labels = ["CRH Simulation", "ACTH Simulation", "Cortisol Simulation", "GR Simulation"], 
                sims_line_colors = ["blue", "blue", "blue", "blue"],
                std_labels = ["Mean +- Standard Deviation", "Mean +- Standard Deviation", "Mean +- Standard Deviation", "Mean +- Standard Deviation"],
                real_data_labels = ["Nelson ACTH - Patient Mean", "Nelson ACTH - Patient Mean", "Nelson Cortisol - Patient Mean"], 
                graph_titles = ["CRH Concentration", "ACTH Concentration", "Cortisol Concentration", "GR Concentration"], 
                xaxis_labels = ["Time (h)", "Time (h)", "Time (h)", "Time (h)"],
                yaxis_labels = ["CRH (micrograms/dL)", "ACTH (pg/mL)", "Cortisol (micrograms/dL)", "GR (micrograms)"],
                legend_locs = ["upper right", "upper right", "upper right", "upper left"],
                figure_size = (25,40),
                savefile=filename_root+".png")

## Parameter Optimization Against Basal Data Sets <a name="basalInstructions" />

Since these data sets have data points over a 24-hour period, rather than 2.35 hours, you will need to change the time interval over which the ODE solver integrates. To do this, go to the cell directly above the heading **Put Raw Data Into Arrays** and change the value of t_end to the following:

    t_end = 24.26

The reason you add the extra 0.26 hours is that you need to make sure that when you interpolate between your simulated data points the line covers every real-world data point so that you don't cause issues when computing the cost function (and the last data point for the Golier cortisol concentration data sets is at 24.25 hours).

After making this change, you need to again change the `data_to_match` list so that you are matching the basal data set in which you are interested.

First, choose which data set you wish to match. Here are the options:

- yehuda.controlCortisol
- yehuda.PTSDCortisol
- yehuda.depressedCortisol
- carroll.controlCortisol & carroll.controlACTH
- carroll.LCDepressedCortisol & carroll.LCDepressedACTH (LC = Low Cortisol)
- carroll.HCDepressedCortisol & carroll.HCDepressedACTH (HC = High Cortisol)
- golier.PTSDCortisol & golierPTSDACTH
- golier.nonPTSDTraumaExposedCortisol & golier.nonPTSDTraumaExposedACTH
- golier.nonPTSDNonExposedCortisol & golier.nonPTSDNonExposedACTH
- bremner.abusedPTSDCortisol
- bremner.nonAbusedPTSDCortisol
- bremner.nonAbusedNonPTSDCortisol

**Note:** To see what any of these data sets looks like, click on the **Plot Basal Data Sets** heading in the Table of Contents.

**Note Also:** These data sets all come in smoothed versions (each data point is set to the average of the nearest 5 points of the unsmoothed data). Also, the data sets by Carroll, Golier and Bremner also come in rearranged (or smoothed & rearranged) versions to match the starting time of the Yehuda data (10AM). To use any of these versions, simply append one of the following tags to the end of the data set name (before the indices): `_smooth`, `_rearr`, or `_rearr_smooth`.

First, I will cover what to do with data sets that contain both ACTH and cortisol values, and then afterwards I will cover using the Yehuda and Bremner data sets (which have only cortisol concentration data). Just as with the Nelson data, in all of these data sets the first column is the time step values. This means that if you take any of these arrays and index it with `[:,0]`, you are referring to the time steps. These are the values we need to use as the first (ACTH time steps) and third (cortisol time steps) indices in the `data_to_match` list.

Then for the second and fourth indices, you index the same data sets with `[:,1]` to mean the second column (which contains the mean concentration values for each patient group).

Here are a couple of examples showing lists you can use for optimization:

- `carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1]`
    - The smoothed Control group mean for the Carroll data set
- `golier.nonPTSDTraumaExposedACTH[:,0], golier.nonPTSDTraumaExposedACTH[:,1], golier.nonPTSDTraumaExposedCortisol[:,0], golier.nonPTSDTraumaExposedCortisol[:,1]`
    - The Trauma-Exposed Control group mean for the Golier data set
    
In order to run simulations against data sets that do not include ACTH concentration data, you will need to change the name of the cost function to `optimize.SSE_cost_noACTH` and then update `data_to_match` to not include the two arguments for ACTH data. To use the Yehuda Control group data, this would look like:

    data_to_match = [yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1]]
    return optimize.SSE_cost_noACTH(data_to_match[0], data_to_match[1], simData)

For data without ACTH concentrations, you will also need to comment out the current definition of `y0` and uncomment the following line (and change the ICs to the desired values for CRH and ACTH):

    #y0 = [0, 10, data_to_match[1][0], 0]
    
At this point, you're ready to run the parameter optimization, so run all of the cells under the heading **Run the Optimization**.

The cell directly under the heading **Save Output to File** should again have the filename changed to something that reflects the data set you're matching now. For instance, the filename root when matching the smoothed Carroll Control group and optimizing ICs would become:

    filename_root = 'sriramModel_output/carroll-control-smooth-5-iterations-ICopt'
            
Finally, the cells under the heading **Plots** should be run again to generate graphs. The same process of giving inputs to the object dialog will be performed and then the method `make_graphs()` should be run with any optional arguments desired.

## Running Without Parameter Optimization <a name="noOptInstructions" />

To reproduce Figures 3 & 4 from the Sriram et al. (2012) paper, you do not need to modify any code. Simply run all cells in the sections labeled **Figure 3 Reproduction** and **Figure 4 Reproduction**.

To run the model with any set of paramaters you desire, without optimization, you can use the cells under the heading **No Optimization Run**. Change the parameters and initial conditions defined under the heading **Parameters and Initial Conditions**, and then run the cell containing the following line:

    data_no_opt = model(authors_params, y0)
    
Then run the cells under **Plots** to create graphs as described in the sections regarding simulations with parameter optimization.

[Back to Top](#top)

# Imports <a name="imports"></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
from scipy.interpolate import interp1d
from tabulate import tabulate
import matplotlib
import pandas as pd
from VeVaPy import DEsolver, optimize
from VeVaPy.dataImport import data
from VeVaPy.visualize import Visualizer

[Back to Top](#top)

# Parameters and Initial Conditions <a name="params"/>

In [ ]:
# Set the Initial Conditions
# This definition will be used for no-optimization simulations, and otherwise will be updated in the section of code
#  labeled Run the Optimization

y0 = [1, 20.71677102294828, 6.314796551724138, 2]

In [ ]:
# initialize parameter values for use when running without parameter optimization
#     These are the optimized values that the authors got from SensSB toolbox

k_stress = 13.7 # normal: 10.1, depressed: 13.7, PTSD: 17.5
k_i = 1.60      # normal: 1.51, depressed: 1.60, PTSD: 1.17
V_S3 = 3.25
K_m1 = 1.74
K_P2 = 8.3
V_S4 = 0.907
K_m2 = 0.112
K_P3 = 0.945
V_S5 = 0.00535
K_m3 = 0.0768
K_d1 = 0.00379
K_d2 = 0.00916
K_d3 = 0.356
n1 = 5.43
n2 = 5.1
K_b = 0.0202
G_tot = 3.28
V_S2 = 0.0509
K1 = 0.645
K_d5 = 0.0854
p_stress = 1

# authors' params without p_stress
authors_params = [k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5]

# with p_stress
#authors_params = [k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5, p_stress]

In [ ]:
# here are the bounds used for all of the parameters, based on the authors' reported bounds
#bounds = [(5,20), (0.5,3), (3,4), (1,2), (7,11), (0.5,1.5), (0.08,2), (0.5,1.2), (0.001,0.008), (0.03,0.08), (0.002,0.005), (0.001,0.01), (0.1,0.5), (4,6), (4,6), (0.008,0.05), (2,5), (0.01,0.07), (0.2,0.7), (0.04,0.09)]

In [ ]:
# wider bounds for fitting other data sets than original
#bounds = [(1,30), (0.1,5), (1,10), (.1,10), (1,31), (0.1,10.5), (0.001,10), (0.1,10), (0.001,0.1), (0.001,0.5), (0.001,0.1), (0.001,0.1), (0.01,5), (1,15), (1,15), (0.001,0.5), (.02,45), (0.0001,7), (0.02,7), (0.004,0.9)]

In [ ]:
# wider bounds WITH p_stress included for Nelson data
#bounds = [(1,30), (0.1,5), (1,10), (.1,10), (1,31), (0.1,10.5), (0.001,10), (0.1,10), (0.001,0.1), (0.001,0.5), (0.001,0.1), (0.001,0.1), (0.01,5), (1,15), (1,15), (0.001,0.5), (.02,45), (0.0001,7), (0.02,7), (0.004,0.9), (1,100)]

In [ ]:
# wider bounds WITHOUT p_stress included for Nelson data
#bounds = [(1,30), (0.1,5), (1,10), (.1,10), (1,31), (0.1,10.5), (0.001,10), (0.1,10), (0.001,0.1), (0.001,0.5), (0.001,0.1), (0.001,0.1), (0.01,5), (1,15), (1,15), (0.001,0.5), (.02,45), (0.0001,7), (0.02,7), (0.004,0.9)]

In [ ]:
# wider bounds WITHOUT p_stress included for Nelson data
#
# first two bounds are for CRH and GR initial conditions, respectively
bounds = [(0, 20), (0,10), (1,30), (0.1,5), (1,10), (.1,10), (1,31), (0.1,10.5), (0.001,10), (0.1,10), (0.001,0.1), (0.001,0.5), (0.001,0.1), (0.001,0.1), (0.01,5), (1,15), (1,15), (0.001,0.5), (.02,45), (0.0001,7), (0.02,7), (0.004,0.9)]

In [ ]:
# time interval and step definition
t_start = -0.01
t_end = 2.35
t_step = 0.01

[Back to Top](#top)

# Put Raw Data Into Arrays <a name="rawdata"/>

In [ ]:
# Create an instance of the data class for each data set contained in the VeVaPy library, and set the time
# scale to hours.
yehuda = data("yehuda", "hours")
carroll = data("carroll", "hours")
golier = data("golier", "hours")
bremner = data("bremner", "hours")
nelson = data("nelson", "hours")

[Back to Top](#top)

## Plot Basal Data Sets <a name="plotBasalData"></a>

In [ ]:
%matplotlib inline
font = {'size'   : 18}
matplotlib.rc('font', **font)

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (20,20))

ax1.plot(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], label = "Control Group Cortisol")
ax1.plot(yehuda.controlCortisol_smooth[:,0], yehuda.controlCortisol_smooth[:,1], label = "Control Group Cortisol - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(yehuda.PTSDCortisol[:,0], yehuda.PTSDCortisol[:,1], label = "PTSD Group Cortisol")
ax2.plot(yehuda.PTSDCortisol_smooth[:,0], yehuda.PTSDCortisol_smooth[:,1], label = "PTSD Group Cortisol - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(yehuda.depressedCortisol[:,0], yehuda.depressedCortisol[:,1], label = "Depression Group Cortisol")
ax3.plot(yehuda.depressedCortisol_smooth[:,0], yehuda.depressedCortisol_smooth[:,1], label = "Depression Group Cortisol - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

font = {'size'   : 20}
matplotlib.rc('font', **font)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (25,25))

ax1.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax1.plot(carroll.HCDepressedCortisol_rearr[:,0], carroll.HCDepressedCortisol_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax1.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax1.plot(carroll.HCDepressedCortisol_rearr_smooth[:,0], carroll.HCDepressedCortisol_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="upper right", shadow = True, fancybox = True)

ax2.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax2.plot(carroll.LCDepressedCortisol_rearr[:,0], carroll.LCDepressedCortisol_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax2.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax2.plot(carroll.LCDepressedCortisol_rearr_smooth[:,0], carroll.LCDepressedCortisol_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="upper right", shadow = True, fancybox = True)

ax3.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax3.plot(carroll.HCDepressedACTH_rearr[:,0], carroll.HCDepressedACTH_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax3.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax3.plot(carroll.HCDepressedACTH_rearr_smooth[:,0], carroll.HCDepressedACTH_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax3.legend(loc="upper right", shadow = True, fancybox = True)

ax4.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax4.plot(carroll.LCDepressedACTH_rearr[:,0], carroll.LCDepressedACTH_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax4.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax4.plot(carroll.LCDepressedACTH_rearr_smooth[:,0], carroll.LCDepressedACTH_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="upper right", shadow = True, fancybox = True)

In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(nrows = 6, figsize = (15,20))

ax1.plot(golier.PTSDCortisol_rearr_smooth[:,0], golier.PTSDCortisol_rearr_smooth[:,1], label = "Trauma Exposed PTSD Cortisol - Smoothed")
ax1.plot(golier.PTSDCortisol_rearr[:,0], golier.PTSDCortisol_rearr[:,1], label = "Trauma Exposed PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,0], golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD Cortisol - Smoothed")
ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr[:,0], golier.nonPTSDTraumaExposedCortisol_rearr[:,1], label = "Trauma Exposed Non-PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(golier.nonPTSDNonExposedCortisol_rearr_smooth[:,0], golier.nonPTSDNonExposedCortisol_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD Cortisol - Smoothed")
ax3.plot(golier.nonPTSDNonExposedCortisol_rearr[:,0], golier.nonPTSDNonExposedCortisol_rearr[:,1], label = "Non-Exposed Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)

ax4.plot(golier.PTSDACTH_rearr_smooth[:,0], golier.PTSDACTH_rearr_smooth[:,1], label = "Trauma Exposed PTSD ACTH - Smoothed")
ax4.plot(golier.PTSDACTH_rearr[:,0], golier.PTSDACTH_rearr[:,1], label = "Trauma Exposed PTSD ACTH")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="lower right", shadow = True, fancybox = True)

ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,0], golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD ACTH - Smoothed")
ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr[:,0], golier.nonPTSDTraumaExposedACTH_rearr[:,1], label = "Trauma Exposed Non-PTSD ACTH")
ax5.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax5.legend(loc="lower right", shadow = True, fancybox = True)

ax6.plot(golier.nonPTSDNonExposedACTH_rearr_smooth[:,0], golier.nonPTSDNonExposedACTH_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD ACTH - Smoothed")
ax6.plot(golier.nonPTSDNonExposedACTH_rearr[:,0], golier.nonPTSDNonExposedACTH_rearr[:,1], label = "Non-Exposed Non-PTSD ACTH")
ax6.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax6.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(bremner.abusedPTSDCortisol_rearr_smooth[:,0], bremner.abusedPTSDCortisol_rearr_smooth[:,1], label = "Abused PTSD Cortisol - Smoothed")
ax1.plot(bremner.abusedPTSDCortisol_rearr[:,0], bremner.abusedPTSDCortisol_rearr[:,1], label = "Abused PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(bremner.nonAbusedPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused PTSD Cortisol - Smoothed")
ax2.plot(bremner.nonAbusedPTSDCortisol_rearr[:,0], bremner.nonAbusedPTSDCortisol_rearr[:,1], label = "Non-Abused PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused Non-PTSD Cortisol - Smoothed")
ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr[:,0], bremner.nonAbusedNonPTSDCortisol_rearr[:,1], label = "Non-Abused Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax3.legend(loc="lower left", shadow = True, fancybox = True)


[Back to Top](#top)

## Plot Nelson Data Sets <a name="plotNelsonData"></a>

In [ ]:
matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

ax1.plot(nelson.ACTH[:,0], nelson.ACTH[:,3], label = "Mean ACTH")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.cortisol[:,0], nelson.cortisol[:,3], label = "Mean Cortisol")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
# graph the Nelson data (only showing one patient at a time currently)
matplotlib.rc('font', **{'size'   : 12})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

#ax1.plot(nelsonACTH[:,0], nelsonACTH[:,1])
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,1], label = "Patient 1")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,2], label = "Patient 2")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,3], label = "Patient 3")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,4], label = "Patient 4")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,5], label = "Patient 5")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,6], label = "Patient 6")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,7], label = "Patient 7")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,8], label = "Patient 8")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,9], label = "Patient 9")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,10], label = "Patient 10")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,11], label = "Patient 11")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,12], label = "Patient 12")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,13], label = "Patient 13")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], label = "Patient 14")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

#ax2.plot(nelsonCortisol[:,0], nelsonCortisol[:,1])
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,1], label = "Patient 1")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,2], label = "Patient 2")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,3], label = "Patient 3")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,4], label = "Patient 4")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,5], label = "Patient 5")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,6], label = "Patient 6")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,7], label = "Patient 7")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,8], label = "Patient 8")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,9], label = "Patient 9")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,10], label = "Patient 10")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,11], label = "Patient 11")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,12], label = "Patient 12")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,13], label = "Patient 13")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14], label = "Patient 14")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
# graph the Nelson data (only showing one patient at a time currently)
matplotlib.rc('font', **{'size'   : 12})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

#ax1.plot(nelsonACTH[:,0], nelsonACTH[:,1])
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], label = "Patient 1")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,2], label = "Patient 2")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,3], label = "Patient 3")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,4], label = "Patient 4")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,5], label = "Patient 5")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,6], label = "Patient 6")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,7], label = "Patient 7")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,8], label = "Patient 8")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,9], label = "Patient 9")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,10], label = "Patient 10")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,11], label = "Patient 11")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,12], label = "Patient 12")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,13], label = "Patient 13")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,14], label = "Patient 14")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

#ax2.plot(nelsonCortisol[:,0], nelsonCortisol[:,1])
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1], label = "Patient 1")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,2], label = "Patient 2")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,3], label = "Patient 3")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,4], label = "Patient 4")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,5], label = "Patient 5")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,6], label = "Patient 6")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,7], label = "Patient 7")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,8], label = "Patient 8")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,9], label = "Patient 9")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,10], label = "Patient 10")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,11], label = "Patient 11")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,12], label = "Patient 12")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,13], label = "Patient 13")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,14], label = "Patient 14")
ax2.legend(loc = "upper left", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
%matplotlib inline

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

ax1.plot(nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], label = "Healthy Control Group", color = "blue")
ax1.plot(nelson.depressedACTH_mean[:,0], nelson.depressedACTH_mean[:,1], label = "Depression Group", color = "orange")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1], label = "Healthy Control Group", color = "blue")
ax2.plot(nelson.depressedCortisol_mean[:,0], nelson.depressedCortisol_mean[:,1], label = "Depression Group", color = "orange")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#plt.savefig("Nelson Data Visualization Figures/nelson-control-vs-depressed-mean-comparison.png", dpi = 300)

In [ ]:
%matplotlib inline

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (25,20))

ax1.plot(nelson.melancholicACTH_mean[:,0], nelson.melancholicACTH_mean[:,1], label = "Melancholic Depression Group", color = "blue")
ax1.plot(nelson.atypicalACTH_mean[:,0], nelson.atypicalACTH_mean[:,1], label = "Atypical Depression Group", color = "orange")
ax1.plot(nelson.neitherACTH_mean[:,0], nelson.neitherACTH_mean[:,1], label = "Non-categorized Depression Group", color = "red")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol_mean[:,1], label = "Melancholic Depression Group", color = "blue")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol_mean[:,1], label = "Atypical Depression Group", color = "orange")
ax2.plot(nelson.neitherCortisol[:,0], nelson.neitherCortisol_mean[:,1], label = "Non-categorized Depression Group", color = "red")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#plt.savefig("Nelson Data Visualization Figures/nelson-depression-subtypes-mean-comparison.png", dpi = 300)

In [ ]:
%matplotlib inline

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (20,20))

ax1.plot(nelson.healthyACTH[:,0], nelson.healthyACTH_mean[:,1], label = "Healthy Control Group", color = "blue")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH_mean[:,1], label = "Melancholic Depression Group", color = "orange")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.healthyCortisol[:,0], nelson.healthyCortisol_mean[:,1], label = "Healthy Control Group", color = "blue")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol_mean[:,1], label = "Melancholic Depression Group", color = "orange")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#plt.savefig("Nelson Data Visualization Figures/nelson-control-vs-melancholic-depression-mean-comparison.png", dpi = 300)

[Back to Top](#top)

# Model Function--Includes ODE Solver <a name="modelfunction"></a>

In [ ]:
def model(params, ics):
    def ode_system(t, y):
        dy = np.zeros(4)
        # parameters to be optimized including p_stress
        #[k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5, p_stress] = params
        
        # parameters to be optimized not including p_stress
        [k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5] = params
        
        # Change whether we use the stress_input function to add a time-dependent stressor by changing which equation
        #  for dy[0] is not commented
        #dy[0] = stress_input(t)*k_stress*(k_i**n2/(k_i**n2 + y[3]**n2)) - V_S3*(y[0]/(K_m1 + y[0])) - K_d1*y[0]
        dy[0] = k_stress*(k_i**n2/(k_i**n2 + y[3]**n2)) - V_S3*(y[0]/(K_m1 + y[0])) - K_d1*y[0]
        dy[1] = K_P2*y[0]*(k_i**n2/(k_i**n2 + y[3]**n2)) - V_S4*(y[1]/(K_m2 + y[1])) - K_d2*y[1]
        dy[2] = K_P3*y[1] - V_S5*(y[2]/(K_m3 + y[2])) - K_d3*y[2]
        dy[3] = K_b*y[2]*(G_tot - y[3]) + V_S2*(y[3]**n1/(K1**n1 + y[3]**n1)) - K_d5*y[3]

        return dy
    
    def stress_input(t):
        #if (0.5 <= t <= 0.51):
        if (0.5 <= t <= 0.83):
            return p_stress
        else:
            return 1
    
    timeSeries = DEsolver.solve(ode_system, ics, t_start, t_step, t_end, ode_steps=1000, ode_atol = 1e-8, ode_rtol = 1e-8)
    return timeSeries

[Back to Top](#top)

# Cost Function Definition <a name="cost"></a>

In [ ]:
def cost_fun(params):
    global y0, data_to_match
    
    # This code will optimize the initial conditions for CRH and GR, uncomment the code block below to not optimize ICs
    y0 = [params[0], y0[1], y0[2], params[1]]
    simData = model(params[2:], y0)
    
    # Uncomment here and comment both lines above to not optimize ICs. (You'll also need to change bounds to not have
    #  bounds included for the ICs)
    #simData = model(params, y0)
    
    # To optimize ACTH as well as CRH and GR (for use with basal data containing only cortisol concentrations), use
    #  the following lines:
    #y0 = [params[0], params[1], y0[2], params[2]]
    #simData = model(params[3:], y0)
    # Note that you'll also need to add a bound for the IC of ACTH in the list of parameter bounds
    
    return optimize.SSE_cost(data_to_match[0], data_to_match[1], data_to_match[2], data_to_match[3], simData)

[Back to Top](#top)

# Run the Optimization <a name="run"></a>

In [ ]:
# Define the data set to match with the parameter optimization algorithm.
# Requires 4 indices, in the order:
# ACTH time steps, ACTH data, Cortisol time steps, Cortisol data
data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]]

# For matching data with only cortisol concentrations, use the following line and change the data sets as desired:
#data_to_match = [yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1]]

In [ ]:
# We define the initial conditions using the real-world data to match defined above.
# We set CRH and GR ICs both to zero to start, but we will be optimizing these

y0 = [0, data_to_match[1][0], data_to_match[3][0], 0]

# For matching data with only cortisol concentrations, use the following line and change the ICs as desired:
#y0 = [0, 10, data_to_match[1][0], 0]

In [ ]:
# We call the run() method of the optimize module, which will run the parameter optimization given the arguments we pass
# Required arguments are the cost function, the model, and the real-world data we want to match
#
# Optional arguments include the initial conditions we want to optimize (if any), the number of iterations to run,
#  the maximum number of optimization steps per iteration of the algorithm, the algorithm to use (defaults to 
#  differential_evolution), and the popsize to use (larger popsize gives more accurate optimization but is more 
#  computationally expensive) 
opt_pars, simData_all = optimize.run(cost_fun, model, data_to_match, y0, bounds, ICopt_indices=[0,3], num_iter=5)


[Back to Top](#top)

# Save Output to File <a name="saveoutput"></a>

In [ ]:
# Change the root filename, this will have the array name appended to it
#  to make the filename of the Excel files, textfiles and graphs
filename_root = "sriramModel_output/nelson-patientMean-5-iterations-ICOpt"

In [ ]:
# Create the pandas DataFrame object for opt_pars
# I've typed out each individual heading for the parameter names that were
#  optimized, and assigned the correct column of opt_pars to them
#
# NOTE: I've been unable to get this code to work for 1 iteration of parameter optimization (only for opt_pars)
# Hopefully in the near future I'll get it worked out, but for some reason I can't get it to be output as a single row
# with 21 columns. It'll only output as a single column with 21 rows.
df_opt_pars = pd.DataFrame(opt_pars, columns=['Cost',
                                              'k_stress',
                                              'k_i',
                                              'V_S3',
                                              'K_m1',
                                              'K_P2',
                                              'V_S4',
                                              'K_m2',
                                              'K_P3',
                                              'V_S5',
                                              'K_m3',
                                              'K_d1',
                                              'K_d2',
                                              'K_d3',
                                              'n1',
                                              'n2',
                                              'K_b',
                                              'G_tot',
                                              'V_S2',
                                              'K1',
                                              'K_d5'])
# Create the pandas DataFrame object for simData_all
# I've typed out each individual heading for the variables and iterations,
# and assigned the correct column of simData_all to them
df_simData_all = pd.DataFrame(simData_all, columns=['Iteration 1 Time',
                                                    'Iteration 1 CRH',
                                                    'Iteration 1 ACTH',
                                                    'Iteration 1 Cortisol',
                                                    'Iteration 1 GR',
                                                    'Iteration 2 Time',
                                                    'Iteration 2 CRH',
                                                    'Iteration 2 ACTH',
                                                    'Iteration 2 Cortisol',
                                                    'Iteration 2 GR',
                                                    'Iteration 3 Time',
                                                    'Iteration 3 CRH',
                                                    'Iteration 3 ACTH',
                                                    'Iteration 3 Cortisol',
                                                    'Iteration 3 GR',
                                                    'Iteration 4 Time',
                                                    'Iteration 4 CRH',
                                                    'Iteration 4 ACTH',
                                                    'Iteration 4 Cortisol',
                                                    'Iteration 4 GR',
                                                    'Iteration 5 Time',
                                                    'Iteration 5 CRH',
                                                    'Iteration 5 ACTH',
                                                    'Iteration 5 Cortisol',
                                                    'Iteration 5 GR'])

# Create an instance of the ExcelWriter class and open the file using a with statement
with pd.ExcelWriter(filename_root+".xlsx") as writer:
    # Define the header format, so that it's bold, text is wrapped, it has a 
    #  colored background and a border
    header_format = writer.book.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    # Write the opt_pars array to a sheet in the file, we skip adding in the headers here and add them with the above
    #  format afterwards. We also change the row index to start at 1, rather than 0.
    df_opt_pars.index = list(range(1,len(opt_pars[:,0])+1))
    df_opt_pars.to_excel(writer, sheet_name="Optimized Parameters", startrow=1, header=False)
    
    # Write the simData_all array to another sheet in the file, we skip adding in the headers here and add them with the above
    #  format afterwards. We also disable the row index numbers, as they are not necessary here.
    df_simData_all.to_excel(writer, sheet_name="Simulated Concentration Data", startrow=1, header=False, index=False)
    
    # Loop through each header in opt_pars DataFrame and write to the sheet with formatting
    for col,val in enumerate(df_opt_pars.columns.values):
        # We write in the sheet "Optimized Parameters" in the first row, starting with the second column 
        #  (because of the row indices), using the headers from the DataFrame and the header format we defined above
        writer.sheets["Optimized Parameters"].write(0, col+1, val, header_format)
    
    # Loop through each header in simData_all DataFrame and write to the sheet with formatting
    for col,val in enumerate(df_simData_all.columns.values):
        # We write in the sheet "Simulated Concentration Data" in the first row, starting with the first column 
        #  (because we turned off the row indices), using the headers from the DataFrame and 
        #  the header format we defined above
        writer.sheets["Simulated Concentration Data"].write(0, col, val, header_format)
    

In [ ]:
# Save the initial conditions and bounds to text files, also.
np.savetxt(filename_root+'-initial-conditions.txt', y0)
np.savetxt(filename_root+'-bounds.txt', bounds)

[Back to Top](#top)

# Compute Means and Std Devations of Parameters and Output as Table <a name="paramtable"></a>

In [ ]:
k_stress_mean = np.mean(opt_pars[:,1])
k_stress_std = np.std(opt_pars[:,1])
k_i_mean = np.mean(opt_pars[:,2])
k_i_std = np.std(opt_pars[:,2])
V_S3_mean = np.mean(opt_pars[:,3])
V_S3_std = np.std(opt_pars[:,3])
K_m1_mean = np.mean(opt_pars[:,4])
K_m1_std = np.std(opt_pars[:,4])
K_P2_mean = np.mean(opt_pars[:,5])
K_P2_std = np.std(opt_pars[:,5])
V_S4_mean = np.mean(opt_pars[:,6])
V_S4_std = np.std(opt_pars[:,6])
K_m2_mean = np.mean(opt_pars[:,7])
K_m2_std = np.std(opt_pars[:,7])
K_P3_mean = np.mean(opt_pars[:,8])
K_P3_std = np.std(opt_pars[:,8])
V_S5_mean = np.mean(opt_pars[:,9])
V_S5_std = np.std(opt_pars[:,9])
K_m3_mean = np.mean(opt_pars[:,10])
K_m3_std = np.std(opt_pars[:,10])
K_d1_mean = np.mean(opt_pars[:,11])
K_d1_std = np.std(opt_pars[:,11])
K_d2_mean = np.mean(opt_pars[:,12])
K_d2_std = np.std(opt_pars[:,12])
K_d3_mean = np.mean(opt_pars[:,13])
K_d3_std = np.std(opt_pars[:,13])
n1_mean = np.mean(opt_pars[:,14])
n1_std = np.std(opt_pars[:,14])
n2_mean = np.mean(opt_pars[:,15])
n2_std = np.std(opt_pars[:,15])
K_b_mean = np.mean(opt_pars[:,16])
K_b_std = np.std(opt_pars[:,16])
G_tot_mean = np.mean(opt_pars[:,17])
G_tot_std = np.std(opt_pars[:,17])
V_S2_mean = np.mean(opt_pars[:,18])
V_S2_std = np.std(opt_pars[:,18])
K1_mean = np.mean(opt_pars[:,19])
K1_std = np.std(opt_pars[:,19])
K_d5_mean = np.mean(opt_pars[:,20])
K_d5_std = np.std(opt_pars[:,20])
#p_stress_mean = np.mean(opt_pars[:,21])
#p_stress_std = np.std(opt_pars[:,21])

In [ ]:
#print(tabulate([["k_stress", "%f +- %f" % (k_stress_mean, k_stress_std)], ["k_i", "%f +- %f" % (k_i_mean, k_i_std)], ["V_S3", "%f +- %f" % (V_S3_mean, V_S3_std)], ["K_m1", "%f +- %f" % (K_m1_mean, K_m1_std)], ["K_P2", "%f +- %f" % (K_P2_mean, K_P2_std)], ["V_S4", "%f +- %f" % (V_S4_mean, V_S4_std)], ["K_m2", "%f +- %f" % (K_m2_mean, K_m2_std)], ["K_P3", "%f +- %f" % (K_P3_mean, K_P3_std)], ["V_S5", "%f +- %f" % (V_S5_mean, V_S5_std)], ["K_m3", "%f +- %f" % (K_m3_mean, K_m3_std)], ["K_d1", "%f +- %f" % (K_d1_mean, K_d1_std)], ["K_d2", "%f +- %f" % (K_d2_mean, K_d2_std)], ["K_d3", "%f +- %f" % (K_d3_mean, K_d3_std)], ["n1", "%f +- %f" % (n1_mean, n1_std)], ["n2", "%f +- %f" % (n2_mean, n2_std)], ["K_b", "%f +- %f" % (K_b_mean, K_b_std)], ["G_tot", "%f +- %f" % (G_tot_mean, G_tot_std)], ["V_S2", "%f +- %f" % (V_S2_mean, V_S2_std)], ["K1", "%f +- %f" % (K1_mean, K1_std)], ["K_d5", "%f +- %f" % (K_d5_mean, K_d5_std)], ["p_stress", "%f +- %f" % (p_stress_mean, p_stress_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))
print(tabulate([["k_stress", "%f +- %f" % (k_stress_mean, k_stress_std)], ["k_i", "%f +- %f" % (k_i_mean, k_i_std)], ["V_S3", "%f +- %f" % (V_S3_mean, V_S3_std)], ["K_m1", "%f +- %f" % (K_m1_mean, K_m1_std)], ["K_P2", "%f +- %f" % (K_P2_mean, K_P2_std)], ["V_S4", "%f +- %f" % (V_S4_mean, V_S4_std)], ["K_m2", "%f +- %f" % (K_m2_mean, K_m2_std)], ["K_P3", "%f +- %f" % (K_P3_mean, K_P3_std)], ["V_S5", "%f +- %f" % (V_S5_mean, V_S5_std)], ["K_m3", "%f +- %f" % (K_m3_mean, K_m3_std)], ["K_d1", "%f +- %f" % (K_d1_mean, K_d1_std)], ["K_d2", "%f +- %f" % (K_d2_mean, K_d2_std)], ["K_d3", "%f +- %f" % (K_d3_mean, K_d3_std)], ["n1", "%f +- %f" % (n1_mean, n1_std)], ["n2", "%f +- %f" % (n2_mean, n2_std)], ["K_b", "%f +- %f" % (K_b_mean, K_b_std)], ["G_tot", "%f +- %f" % (G_tot_mean, G_tot_std)], ["V_S2", "%f +- %f" % (V_S2_mean, V_S2_std)], ["K1", "%f +- %f" % (K1_mean, K1_std)], ["K_d5", "%f +- %f" % (K_d5_mean, K_d5_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))


In [ ]:
np.savetxt(filename_root+'-param-means-std-devs.txt', [k_stress_mean, k_stress_std, k_i_mean, k_i_std, V_S3_mean, V_S3_std, K_m1_mean, K_m1_std, K_P2_mean, K_P2_std, V_S4_mean, V_S4_std, K_m2_mean, K_m2_std, K_P3_mean, K_P3_std, V_S5_mean, V_S5_std, K_m3_mean, K_m3_std, K_d1_mean, K_d1_std, K_d2_mean, K_d2_std, K_d3_mean, K_d3_std, n1_mean, n1_std, n2_mean, n2_std, K_b_mean, K_b_std, G_tot_mean, G_tot_std, V_S2_mean, V_S2_std, K1_mean, K1_std, K_d5_mean, K_d5_std])
#np.savetxt('sriramModel_output/sriramModel-nelson-atypical-patientID_1-param-means-stds-5-iterations-p-stress.txt', [k_stress_mean, k_stress_std, k_i_mean, k_i_std, V_S3_mean, V_S3_std, K_m1_mean, K_m1_std, K_P2_mean, K_P2_std, V_S4_mean, V_S4_std, K_m2_mean, K_m2_std, K_P3_mean, K_P3_std, V_S5_mean, V_S5_std, K_m3_mean, K_m3_std, K_d1_mean, K_d1_std, K_d2_mean, K_d2_std, K_d3_mean, K_d3_std, n1_mean, n1_std, n2_mean, n2_std, K_b_mean, K_b_std, G_tot_mean, G_tot_std, V_S2_mean, V_S2_std, K1_mean, K1_std, K_d5_mean, K_d5_std, p_stress_mean, p_stress_std])


[Back to Top](#top)

# Plots <a name="plots"></a>

In [ ]:
# Create an instance of the Visualizer class, which will start a series of prompts
grapher = Visualizer(globals())

In [ ]:
# The Visualizer method make_graphs can be called without arguments to create simple graphs without labels and with
#  default size and colors
grapher.make_graphs()

In [ ]:
# However, many arguments can also be specified to add detail to the graphs
grapher.make_graphs(std_dev = True, 
                    sims_line_labels = ["CRH Simulation", "ACTH Simulation", "Cortisol Simulation", "GR Simulation"], 
                    sims_line_colors = ["blue", "blue", "blue", "blue"],
                    std_labels = ["Mean +- Standard Deviation", "Mean +- Standard Deviation", "Mean +- Standard Deviation", "Mean +- Standard Deviation"],
                    real_data_labels = ["Nelson ACTH - Patient Mean", "Nelson ACTH - Patient Mean", "Nelson Cortisol - Patient Mean"], 
                    graph_titles = ["CRH Concentration", "ACTH Concentration", "Cortisol Concentration", "GR Concentration"], 
                    xaxis_labels = ["Time (h)", "Time (h)", "Time (h)", "Time (h)"],
                    yaxis_labels = ["CRH (micrograms/dL)", "ACTH (pg/mL)", "Cortisol (micrograms/dL)", "GR (micrograms)"],
                    legend_locs = ["upper right", "upper right", "upper right", "upper left"],
                    figure_size = (25,40),
                    savefile=filename_root+".png")


[Back to Top](#top)

# Figure 3 Reproduction <a name="fig3"></a>

In [ ]:
# initialize parameter values that stay the same between all three groups
#     These are the optimized values that the authors got from SensSB toolbox

k_stress = 10.1
k_i = 1.51
V_S3 = 3.25
K_m1 = 1.74
K_P2 = 8.3
V_S4 = 0.907
K_m2 = 0.112
K_P3 = 0.945
V_S5 = 0.00535
K_m3 = 0.0768
K_d1 = 0.00379
K_d2 = 0.00916
K_d3 = 0.356
n1 = 5.43
n2 = 5.1
K_b = 0.0202
G_tot = 3.28
V_S2 = 0.0509
K1 = 0.645
K_d5 = 0.0854
p_stress = 1

authors_params_control = [k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5]#, p_stress]


In [ ]:
# initialize parameter values that stay the same between all three groups
#     These are the optimized values that the authors got from SensSB toolbox

k_stress = 13.7
k_i = 1.6
V_S3 = 3.25
K_m1 = 1.74
K_P2 = 8.3
V_S4 = 0.907
K_m2 = 0.112
K_P3 = 0.945
V_S5 = 0.00535
K_m3 = 0.0768
K_d1 = 0.00379
K_d2 = 0.00916
K_d3 = 0.356
n1 = 5.43
n2 = 5.1
K_b = 0.0202
G_tot = 3.28
V_S2 = 0.0509
K1 = 0.645
K_d5 = 0.0854
p_stress = 1

authors_params_depressed = [k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5]#, p_stress]


In [ ]:
# initialize parameter values that stay the same between all three groups
#     These are the optimized values that the authors got from SensSB toolbox

k_stress = 17.5
k_i = 1.17
V_S3 = 3.25
K_m1 = 1.74
K_P2 = 8.3
V_S4 = 0.907
K_m2 = 0.112
K_P3 = 0.945
V_S5 = 0.00535
K_m3 = 0.0768
K_d1 = 0.00379
K_d2 = 0.00916
K_d3 = 0.356
n1 = 5.43
n2 = 5.1
K_b = 0.0202
G_tot = 3.28
V_S2 = 0.0509
K1 = 0.645
K_d5 = 0.0854
p_stress = 1

authors_params_ptsd = [k_stress, k_i, V_S3, K_m1, K_P2, V_S4, K_m2, K_P3, V_S5, K_m3, K_d1, K_d2, K_d3, n1, n2, K_b, G_tot, V_S2, K1, K_d5]#, p_stress]


In [ ]:
y0_control = [1, 5, 5, 2]
y0_depressed = [1, 5, 5, 2]
y0_ptsd = [1, 5, 5, 2]

t_start = -0.01
t_end = 300
t_step = 0.01

In [ ]:
%%time

no_opt_control = model(authors_params_control, y0_control)
no_opt_depressed = model(authors_params_depressed, y0_depressed)
no_opt_ptsd = model(authors_params_ptsd, y0_ptsd)

sims_gr_control = no_opt_control[:,4]
sims_cort_control = no_opt_control[:,3]
sims_acth_control = no_opt_control[:,2]
sims_crh_control = no_opt_control[:,1]

sims_gr_depressed = no_opt_depressed[:,4]
sims_cort_depressed = no_opt_depressed[:,3]
sims_acth_depressed = no_opt_depressed[:,2]
sims_crh_depressed = no_opt_depressed[:,1]

sims_gr_ptsd = no_opt_ptsd[:,4]
sims_cort_ptsd = no_opt_ptsd[:,3]
sims_acth_ptsd = no_opt_ptsd[:,2]
sims_crh_ptsd = no_opt_ptsd[:,1]

In [ ]:
%matplotlib inline
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,20))

ax1.plot(np.arange(0, 70, 0.01), sims_cort_control[-7800:-800], label = "CORT sim - Control")
ax1.set(ylabel = "CORT (micrograms/dL)", xlabel = "Time (h)", title = "CORT sim - Control")
#ax1.legend(loc = "upper right", shadow = True, fancybox = True)

ax2.plot(np.arange(0, 70, 0.01), sims_cort_depressed[-8800:-1800], label = "CORT sim - Depression")
ax2.set(ylabel = "CORT (micrograms/dL)", xlabel = "Time (h)", title = "CORT sim - Depression")
#ax2.legend(loc = "upper right", shadow = True, fancybox = True)

ax3.plot(np.arange(0, 70, 0.01), sims_cort_ptsd[-9100:-2100], label = "CORT sim - PTSD")
ax3.set(ylabel = "CORT (micrograms/dL)", xlabel = "Time (h)", title = "CORT sim - PTSD")
#ax3.legend(loc = "upper right", shadow = True, fancybox = True)

plt.savefig("sriramModel_output/sriramModel-figure3-reproduction.png", dpi=300)

[Back to Top](#top)

# Figure 4 Reproduction <a name="fig4"></a>

In [ ]:
# create Fig 4 of free glucocorticoid receptor level
t_end = 240.01

normal = model(authors_params_control, y0)
PTSD = model(authors_params_ptsd, y0)
depressed = model(authors_params_depressed, y0)

G_normal = G_tot - normal[:,4]
G_PTSD = G_tot - PTSD[:,4]
G_depressed = G_tot - depressed[:,4]

plt.plot(normal[-12000:,0], G_normal[-12000:], label='Normal')
plt.plot(PTSD[-12000:,0], G_PTSD[-12000:], label='PTSD')
plt.plot(depressed[-12000:,0], G_depressed[-12000:], label='Depressed')
plt.ylim(0,2.5)
plt.xlabel('t (hours)')
plt.ylabel('G')
plt.title("Figure 4 Reproduction")
plt.legend(loc = "upper right", shadow = True, fancybox = True)
plt.savefig("sriramModel_output/sriramModel-figure4-reproduction.png", dpi=300)

[Back to Top](#top)

# No Optimization Run <a name="no-opt"></a>

In [ ]:
# Define initial conditions
y0 = [5, 20, 6, 1.25]

In [ ]:
%%time # Output the time it takes to run the simulation

# Use the desired parameters and initial conditions to run the model
data_no_opt = model(authors_params, y0)

In [ ]:
df_noOpt = pd.DataFrame(data_no_opt, columns=['Time',
                                              'CRH',
                                              'ACTH',
                                              'Cortisol',
                                              'GR'])

with pd.ExcelWriter("sriramModel_output/no-optimization-run.xlsx") as writer:
    df_noOpt.to_excel(writer, sheet_name="No Optimization Run 1", index=False)

[Back to Top](#top)

# Run Optimizations for Multiple Patients <a name="runMultiple" />

In [ ]:
# Enter the patient number to start and end the optimization loop
start_patient = 1
end_patient = 10

for patient in range(start_patient, end_patient+1):
    print(f"\033[1mCurrent Patient: #{patient}\033[0m")
    # Change the data set to cycle through patients in here. Will only work for Nelson TSST data, including subtype
    #  data sets (as the basal data sets are only mean concentrations, not individual patients)
    data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,patient+1], nelson.cortisol[:,0], nelson.cortisol[:,patient+1]]
    y0 = [1, data_to_match[1][0], data_to_match[3][0], 1]
    
    opt_pars_tmp, simData_all_tmp = optimize.run(cost_fun, model, data_to_match, y0, bounds, ICopt_indices=[0,3], num_iter=5)
    
    if patient == start_patient:
        simData_all_multiple = simData_all_tmp
        opt_pars_multiple = opt_pars_tmp
    else:
        simData_all_multiple = np.hstack((simData_all_multiple, simData_all_tmp))
        opt_pars_multiple = np.hstack((opt_pars_multiple, opt_pars_tmp))
        

[Back to Top](#top)

# Checking Parameter Sets Generated Externally <a name="checkexternal"></a>

In [ ]:
cort_data = np.zeros((2403,len(loaded_params)))
for i in range(len(loaded_params)):
    cort_data[:,i] = np.transpose(np.hstack((i+1,model(loaded_params[i,1:], y0)[:,3])))

In [ ]:
time_data = np.zeros((2403,len(loaded_params)))
for i in range(len(loaded_params)):
    time_data[:,i] = np.transpose(np.hstack((i+1,model(loaded_params[i,1:], y0)[:,0])))

In [ ]:
np.savetxt("sriram-cort-data-yehudaControl-1000-all-params.txt", cort_data)

In [ ]:
np.savetxt("sriram-time-data-yehudaControl-1000-all-params.txt", time_data)

In [ ]:
sim = model(yin_params[next_max,1:], y0)

In [ ]:
sim = model(testing_params, y0)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))
ax1.plot(sim[:,0], sim[:,3], label = "Simulation")
ax1.plot(yehudaControlCortisol[:,0], yehudaControlCortisol[:,1], label = "Yehuda Control Group")
ax1.set(xlabel = "Time", ylabel = "Cortisol", title = "Control Group: Raw Data vs Authors' Optimized Parameters")
ax2.plot(sim[:,0], sim[:,1])
ax2.set(xlabel = "Time", ylabel = "CRH", title = "CRH w/ Authors' Control Group Parameters")
ax3.plot(sim[:,0], sim[:,2])
ax3.set(xlabel = "Time", ylabel = "ACTH", title = "ACTH w/ Authors' Control Group Parameters")

plt.savefig("testing-params-kb-0.01.pdf", dpi = 300)

[Back to Top](#top)

# Load Simulated Concentration Data from Files and Plot <a name="load-and-plot"></a>

In [ ]:
sims_t = np.genfromtxt('sriramModel_output/sriramModel-nelson-atypical-patientID_19-timeSteps-5-iterations.txt')
sims_gr = np.genfromtxt('sriramModel_output/sriramModel-nelson-atypical-patientID_19-sims-gr-5-iterations.txt')
sims_cort = np.genfromtxt('sriramModel_output/sriramModel-nelson-atypical-patientID_19-sims-cort-5-iterations.txt')
sims_crh = np.genfromtxt('sriramModel_output/sriramModel-nelson-atypical-patientID_19-sims-crh-5-iterations.txt')
sims_acth = np.genfromtxt('sriramModel_output/sriramModel-nelson-atypical-patientID_19-sims-acth-5-iterations.txt')

In [ ]:
%matplotlib inline
# plot of raw data with simulation data for cortisol, then just simulation data for crh and acth

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, figsize=(24,40))

#ax1.plot(optimizedSimData[:,0], sims_crh, label = "Mean Simulated CRH")
# for more than 1 iteration, uncomment below
ax1.plot(sims_t, np.mean(sims_crh, axis = 1), label = "Mean Simulated CRH", color = "blue")
ax1.fill_between(sims_t, np.mean(sims_crh, axis = 1) - np.std(sims_crh, axis = 1), np.mean(sims_crh, axis = 1) + np.std(sims_crh, axis = 1), alpha = 0.3, label = "Simulated CRH Standard Deviation")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "CRH (micrograms/dL)", title = "CRH Concentration")

#ax2.plot(optimizedSimData[:,0], sims_acth, label = "Mean")
ax2.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,6], label = "Nelson Atypical MDD ACTH Data - Patient ID 19", color = "orange")
# for more than 1 iteration, uncomment below
ax2.plot(sims_t, np.mean(sims_acth, axis = 1), label = "Mean Simulated ACTH", color = "blue")
ax2.fill_between(sims_t, np.mean(sims_acth, axis = 1) - np.std(sims_acth, axis = 1), np.mean(sims_acth, axis = 1) + np.std(sims_acth, axis = 1), alpha = 0.3, label = "Simulated ACTH Standard Deviation")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

#ax3.plot(optimizedSimData[:,0], sims_cort, label = "Simulation Mean")
ax3.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,6], label = "Nelson Atypical MDD Cortisol Data - Patient ID 19", color = "orange")
# for more than 1 iteration, uncomment below
ax3.plot(sims_t, np.mean(sims_cort, axis = 1), label = "Mean Simulated Cortisol", color = "blue")
ax3.fill_between(sims_t, np.mean(sims_cort, axis = 1) - np.std(sims_cort, axis = 1), np.mean(sims_cort, axis = 1) + np.std(sims_cort, axis = 1), alpha = 0.3, label = "Simulated Cortisol Standard Deviation")
ax3.legend(loc = "lower right", shadow = True, fancybox = True)
ax3.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#ax4.plot(optimizedSimData[:,0], sims_gr, label = "Mean")
# for more than 1 iteration, uncomment below
ax4.plot(sims_t, np.mean(sims_gr, axis = 1), label = "Mean Simulated GR")
ax4.fill_between(sims_t, np.mean(sims_gr, axis = 1) - np.std(sims_gr, axis = 1), np.mean(sims_gr, axis = 1) + np.std(sims_gr, axis = 1), alpha = 0.5, label = "Simulated GR Standard Deviation")
ax4.legend(loc = "upper left", shadow = True, fancybox = True)
ax4.set(xlabel = "Time (h)", ylabel = "GR (micrograms)", title = "GR Concentration")

txt = "cost = [0.759431, 0.84753]"
plt.figtext(0.1, 0.09, txt, wrap = True, horizontalalignment = 'center', fontsize = 30)
#plt.savefig("sriramModel-nelson-atypical-patientID_1-5-iterations-all-params-normalizedCost.png", dpi = 300)
# include p_stress in filename to indicate that we used stress_input function
plt.savefig("sriramModel_output/sriramModel-nelson-atypical-patientID_19-5-iterations-all-params-normalizedCost.png", dpi = 300)


[Back to Top](#top)

# Dependencies <a name="dependencies"></a>

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions